In [4]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Các bước thực hiện
# 1. Đọc file tests training & label
# 2. Chia 3, lấy phần giữa (phần ổn định)
# 3. Phân khung 30ms, dịch khung 20ms, khung chữ nhật
# 4. Tính MFCC trên từng khung => trung bình MFCC trên tất cả các khung => đặc trưng của 1 audio
# 5. Tính trung bình đặc trưng của 21 audio của mỗi nguyên âm => đặc trưng nguyên âm
#### Lưu lại các vector hệ số
# 6. Đọc file input test
# 7. Chia 3 lấy phần giữa
# 8. Phân khung 30ms, dịch khung 20ms, khung chữ nhật

# functions ---------------------
def get_stable_audio(filepath): # trả về phần ổn định ở chính giữa của audio
    audio, sr = librosa.load(filepath, sr=None)
    div_part = len(audio) // 3
    center_part = audio[div_part:2*div_part]
    # chuẩn hoá về 0-1
    center_part = center_part/(max(abs(center_part)))
    return center_part, sr

def windowing(audio, sr, frame_length = 0.03, frame_hop = 0.02): # windowing với length, hop
    
    frame_size = int(frame_length*sr)
    hop_size = int(frame_hop*sr)

    frames = []
    for i in range(0,len(audio),hop_size):
        if i+frame_size < len(audio):
            frames.append(audio[i:i+frame_size])
    return frames

def mfcc(frame,N,sr): # tính N đặc trưng MFCC trên frame, viết gọn của librosa
    return librosa.feature.mfcc(y=frame,sr=sr,n_mfcc=N)

# Đọc input
test_path = "trimmed_signals\\HuanLuyen-16k\\"

input = []
N_MFCC = 13

for file in sorted(os.listdir(test_path)):
    # print(os.path.join(path,file))
        sub_path = os.path.join(test_path,file)
        for sub_file in sorted(os.listdir(sub_path)):
            exact_test_path = os.path.join(sub_path,sub_file)
            label = sub_file.split(".")[0]

            audio, sr = get_stable_audio(exact_test_path)
            frames = windowing(audio, sr)
            MFCCs = []
            for frame in frames:
                MFCCs.append(mfcc(frame,N_MFCC,sr))
            feature = np.average(MFCCs,axis=0)
            feature = feature/max(abs(feature))
            input.append([feature, label])

# input = np.array(input)

c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=480
  warnings.warn(


In [5]:
# check input
# print(input.shape)
# print(input)
# grouping by labels
features = {'a': [], 'e':[], 'i':[], 'o':[], 'u':[]}
for j in input:
    features[j[1]].append(j[0])
print(features['a'])
print(len(features['a']))

[array([[-0.46253866],
       [ 1.        ],
       [-0.57859045],
       [-0.47896686],
       [ 0.16712427],
       [ 0.07286719],
       [-0.36138573],
       [ 0.16583385],
       [-0.04684381],
       [ 0.2866393 ],
       [-0.11105986],
       [ 0.13067496],
       [-0.02520929]], dtype=float32), array([[-0.28788352],
       [ 1.        ],
       [-0.686278  ],
       [ 0.2179706 ],
       [-0.05132259],
       [ 0.01060002],
       [ 0.05118635],
       [-0.10919782],
       [-0.08035276],
       [-0.08611406],
       [-0.05224352],
       [ 0.08853779],
       [-0.0243414 ]], dtype=float32), array([[-0.49220487],
       [ 0.76918286],
       [-1.        ],
       [-0.13535392],
       [-0.10695917],
       [-0.00388006],
       [-0.1382695 ],
       [ 0.02081476],
       [ 0.2090106 ],
       [ 0.06122307],
       [ 0.00238286],
       [-0.04677504],
       [-0.14891146]], dtype=float32), array([[-0.52766424],
       [ 1.        ],
       [-0.38719425],
       [-0.40327138],
  

In [6]:
average_features = {
    'a': np.average(features['a'],axis=0), 
    'e': np.average(features['e'],axis=0), 
    'i': np.average(features['i'],axis=0), 
    'o': np.average(features['o'],axis=0), 
    'u': np.average(features['u'],axis=0)
    }
# print(average_features['a'])
rows = []
for key, f in average_features.items():
    rows.append([key, f.reshape((N_MFCC,))])
df = pd.DataFrame(rows)
df.to_csv('average_features_MFCC_13.csv',index=False)

In [22]:
#cài đặt Kmean lên tập features của từng nguyên âm, lấy tập centroids làm tập đặc trưng cho nguyên âm đó

#chọn ngẫu nhiên K hàng trong X làm tâm
def init_K(X, K):
    return X[np.random.choice(X.shape[0], K, replace=False)]

#duyệt tất cả các record và label tâm
def clustering(X, centroids):
    N = X.shape[0]
    label = np.zeros((1,N))
    for i in range (N):
        xi = X[i,:]
        min = np.linalg.norm(xi - centroids[0])
        for index, c in enumerate(centroids):
            if np.linalg.norm(xi - c) < min:
                min = np.linalg.norm(xi - c)
                label[0,i] = index
    return label

def cluster_of_same_label(index, label, X):
    selected = []
    for i in range(label.shape[1]):
        if label[0,i] == index:
            selected.append(X[i])
    return np.array(selected)

def isEqual(last_label, label):
    for i in range (label.shape[1]):
        if last_label[0,i] != label[0,i]:
            return False
    return True

def K_means(X, K, loop = 20):
    count = 0
    centroids = init_K(X,K)
    last_label = np.zeros((1,X.shape[0]))
    while (count < loop):
        label = clustering(X, centroids)
        #kiểm tra, nếu tập label không đổi => thoát
        if isEqual(last_label, label):
            return centroids, label
        new_centroids = np.zeros(centroids.shape)
        for index in range (centroids.shape[0]):
            new_centroids[index] = np.mean(cluster_of_same_label(index,label,X), axis = 0)
        centroids = new_centroids.copy()
        last_label = label
        count+=1
    return centroids, label

K = 5 # 2 3 4 5
dim = N_MFCC
Kmeans_features = {
    'a': K_means(np.array(features['a']),K)[0], 
    'e': K_means(np.array(features['e']),K)[0],
    'i': K_means(np.array(features['i']),K)[0], 
    'o': K_means(np.array(features['o']),K)[0], 
    'u': K_means(np.array(features['u']),K)[0]
    }
print(Kmeans_features['a'])
print(Kmeans_features['a'].shape)

[[[-0.4701784 ]
  [ 0.96702617]
  [-0.73318392]
  [-0.17632842]
  [-0.00310871]
  [ 0.00238327]
  [-0.00329959]
  [ 0.00958379]
  [ 0.02539909]
  [ 0.04734385]
  [-0.04821149]
  [ 0.00299877]
  [-0.09929143]]

 [[-0.88644081]
  [ 1.        ]
  [-0.23892164]
  [-0.60868156]
  [-0.23871081]
  [ 0.34820724]
  [-0.23911011]
  [ 0.15900186]
  [-0.00898551]
  [ 0.10287116]
  [-0.11283765]
  [-0.02033436]
  [-0.15419574]]

 [[-0.70698792]
  [ 0.88138306]
  [-0.97080094]
  [-0.42555994]
  [-0.2572785 ]
  [ 0.32577667]
  [-0.22343823]
  [ 0.1233509 ]
  [ 0.04006177]
  [ 0.04436912]
  [-0.05919395]
  [-0.04261786]
  [-0.15386903]]

 [[-0.44833308]
  [ 1.        ]
  [-0.42209998]
  [-0.33261347]
  [-0.32574302]
  [ 0.15642688]
  [-0.11122259]
  [ 0.08750949]
  [ 0.05937313]
  [ 0.11466397]
  [-0.03716533]
  [-0.03837099]
  [-0.14206207]]

 [[-0.46253866]
  [ 1.        ]
  [-0.57859045]
  [-0.47896686]
  [ 0.16712427]
  [ 0.07286719]
  [-0.36138573]
  [ 0.16583385]
  [-0.04684381]
  [ 0.2866393 ]


In [23]:
#đọc dữ liệu kiểm thử
test_path = "trimmed_signals\\KiemThu-16k\\"

tests = []

for file in sorted(os.listdir(test_path)):
    # print(os.path.join(path,file))
        sub_path = os.path.join(test_path,file)
        for sub_file in sorted(os.listdir(sub_path)):
            exact_test_path = os.path.join(sub_path,sub_file)
            audio, sr = get_stable_audio(exact_test_path)
            label = sub_file.split(".")[0]
            frames = windowing(audio, sr)
            MFCCs = []
            for frame in frames:
                MFCCs.append(mfcc(frame,N_MFCC,sr))
            feature = np.average(MFCCs,axis=0)
            feature = feature/max(abs(feature))
            tests.append([feature,label]) # label này sẽ được sử dụng để kiểm tra kết quả nhận dạng

# tests = np.array(tests)
print(tests)

[[array([[-0.31302035],
       [ 0.68187904],
       [-1.        ],
       [-0.33267853],
       [-0.7198858 ],
       [ 0.28265962],
       [-0.29246414],
       [-0.0112186 ],
       [ 0.18357766],
       [ 0.30223426],
       [-0.04454084],
       [-0.16344894],
       [ 0.02461572]], dtype=float32), 'a'], [array([[-0.21946636],
       [ 0.6683923 ],
       [-1.        ],
       [ 0.07784884],
       [-0.86411744],
       [-0.12732425],
       [-0.81754404],
       [ 0.3840172 ],
       [ 0.2575701 ],
       [-0.00915566],
       [ 0.12922633],
       [ 0.31886914],
       [-0.26361844]], dtype=float32), 'e'], [array([[-0.17887959],
       [ 1.        ],
       [-0.04408625],
       [ 0.82559174],
       [-0.52209324],
       [-0.48534364],
       [-0.31200624],
       [-0.18877965],
       [-0.53792053],
       [-0.06632993],
       [ 0.14555494],
       [-0.2272473 ],
       [-0.0950537 ]], dtype=float32), 'i'], [array([[-0.5529755 ],
       [ 1.        ],
       [-0.62638736],
  

In [24]:
def classify(test_vectors, features):
    label_result = ''
    min_dist = 1e6
    for key, feature_set in features.items():
        for f in feature_set:
            dist = np.linalg.norm(test_vectors - f)
            if dist < min_dist:
                min_dist = dist
                label_result = key
    return label_result

count = 0
keys = {'a':0, 'e':1, 'i':2, 'o':3, 'u':4}
table = np.zeros((5,5))

for test in tests:
    label_result = classify(test[0], Kmeans_features)
    if (label_result == test[1]):
        count+=1
    table[keys[label_result],keys[test[1]]]+=1
    # print("Label: " + str(test[1]) + ", classified label: " + str(label_result))
print(count)
# hiển thị bảng đúng sai
# từng hàng là label do chương trình nhận dạng
# từng cột là label đúng của dữ liệu
print(table)

90
[[20.  2.  0.  0.  0.]
 [ 1. 15.  1.  0.  0.]
 [ 0.  1. 18.  0.  3.]
 [ 0.  3.  0. 19.  0.]
 [ 0.  0.  2.  2. 18.]]
